In [ ]:
from dotenv import load_dotenv

load_dotenv("../.env")

In [3]:
from llama_index import SimpleDirectoryReader
from llama_index.embeddings import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")
documents = SimpleDirectoryReader("./data/").load_data()

In [16]:
import chromadb
from llama_index import ServiceContext, StorageContext, VectorStoreIndex
from llama_index.vector_stores import ChromaVectorStore
from llama_index.llms import OpenAI

remote_db = chromadb.HttpClient(host="127.0.0.1", port="8000")
chroma_collection = remote_db.get_or_create_collection("quickstart")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

llm = OpenAI(model="gpt-4", temperature=0.1)
service_context = ServiceContext.from_defaults(embed_model=embed_model, llm=llm)
index = VectorStoreIndex.from_documents(
    documents=documents, storage_context=storage_context, service_context=service_context
)

In [17]:
query_engine = index.as_query_engine()
ans = query_engine.query("How to choose appropriate LLM? Answer in detail as much as possible.")

In [ ]:
print(ans.response)

In [19]:
import openai

completion = openai.chat.completions.create(
    model="gpt-4",
    messages=[{"role": "user", "content": "How to choose appropriate LLM? Answer in detail as much as possible."}],
)

In [ ]:
print(completion.choices[0].message.content)

In [ ]:
documents[8].get_content()